# Morphological analysis

- create list of 50 signs and their synonyms from the course
- find a way to group two words into one sign
- read about Morphological analysis to make the pairs to solve the problem of verbs & nouns
- find way to apply rules like removing  على
- find difference between كتب which is book and wrote 




# very important

- كلمة بيتك وبيتى او اى كلمة فيها ملكية ف وقتها حولها للكلمة الاصلية  اللى هيا هنا هتكون بيت وبعدين اعمل اشارة ملكية او اشارة للمخاطب اللى هيا بتكون انك بتشاور على نفسك او بتشاور امامك وايدك مقفولة
- المذكر والمؤنث فى كلمات عديدة يتم باستعمال اشارة الكلمة نفسها تليها اشارة لتحديد المذكر او المؤنث
- some words don't follow the previous rule so i have to seperate between the words that follow these rules and the words that don't
- اشارة اخت عبارة عن اشارة اخ + اشارة امرأة ونفس الامر لكمات عديدة
- فكرة الملكية بنتطبق برضو على الجمع انة اشارة الكلمة وبعدين جمع او لا 


# try to translate these examples
- السلام عليكم ورحمة الله وبركاتة
- انت اسمك اية 
- ما هو اسمك
- كيف حالك
- انت عامل اية 
- ازيك
- ما شاء الله
- الحمد لله
- انت اصم 
- انا شوفتك 
- انت بتعلمنى كثير ربنا يحفظك
- ما هى وظيفتك
- انت بتشتغل اية 
- عندك كام سنة 
- انت متزوج ؟ 

# convert symbols to actual arabic

In [4]:
import re
import os
import json
from collections import defaultdict

In [19]:
sign_language_data = os.path.join("signLanguageData","تعارف.json")


with open(sign_language_data, 'r') as json_file:
    json_data = json.load(json_file)
    for key in json_data:
        print(f' "{key}":{json_data[key]}',",",sep="")
    

 "sign_0":{'words': ['تعارف'], 'description': ''},
 "sign_1":{'words': ['السلام', 'سلام', 'مرحبا', 'ترحيب'], 'description': ''},
 "sign_2":{'words': ['عليكم', 'لكم'], 'description': ''},
 "sign_3":{'words': ['رحمه'], 'description': ''},
 "sign_4":{'words': ['الله'], 'description': ''},
 "sign_5":{'words': ['بركاته', 'بركة'], 'description': ''},
 "sign_6":{'words': ['عامل', 'حالك'], 'description': ''},
 "sign_7":{'words': ['اية', 'كيف'], 'description': ''},
 "sign_8":{'words': ['الحمد', 'الشكر', 'شكر'], 'description': ''},
 "sign_9":{'words': ['تمام', 'جيد'], 'description': ''},
 "sign_10":{'words': ['صباح'], 'description': ''},
 "sign_11":{'words': ['مساء'], 'description': ''},
 "sign_12":{'words': ['الخير', 'حير'], 'description': ''},
 "sign_13":{'words': ['صحتك', 'صحة'], 'description': ''},
 "sign_14":{'words': ['اسمك', 'اسم'], 'description': ''},
 "sign_15":{'words': ['سؤال', 'سأل'], 'description': ''},
 "sign_16":{'words': ['جواب', 'اجاب'], 'description': ''},
 "sign_17":{'words': 

# text processing

In [19]:
import re
import os
from collections import defaultdict


# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","_","-","/",".","،"," و ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!','  ']
    replace = ["ا","ا","ا"," "," ","","",""," و","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ',' ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for a,b in zip(search,replace):
        text = text.replace(a,b)
    
    #trim    
    text = text.strip()
    return [word.strip() for word in text.split(" ")]

    return text

# lemmatization

In [2]:
#https://farasa.qcri.org/lemmatization/

import json
import requests

URL = 'https://farasa.qcri.org/webapi/lemmatization/'
API_KEY = "mgmfGsWLmIFJznbMaC"
LEMMATIZATION_FILE = "lemmatization.json"

with open(LEMMATIZATION_FILE, 'r') as json_file:
    lemmatization_table = json.load(json_file)


def fetch(arr):    
    text = " ".join(arr)
    payload = {'text': text, 'api_key': API_KEY}
    data = requests.post(URL, data=payload)
    result = json.loads(data.text)
    return result['text']

def lemmatization(arr):
    words = []
    for word in arr:
        if(word not in lemmatization_table):
            words.append(word)
    if(len(words)>0):
        lemmatizated_words = fetch(words)
        for original,lemmaitaized in zip(words,lemmatizated_words):
            lemmatization_table[original] = lemmaitaized
    
    
        with open(LEMMATIZATION_FILE, 'w') as write:
            json.dump(lemmatization_table,write)
    
    final_results = []
    for word in arr:
        final_results.append(lemmatization_table[word])
    
    return final_results

# synonyms

In [37]:
import os
import json

sign_language_words = os.path.join("signLanguageWords","تعارف.txt")
sign_language_data = os.path.join("signLanguageData","تعارف.json")

json_data = {}
existing_words = {}
counter = 0;

with open(sign_language_data, 'r') as json_file:
    json_data = json.load(json_file)
    counter = len(json_data)
    for key,value in json_data.items():
        existing_words[value['word']] = 1;


    
with open(sign_language_words,"r") as f:
    lis = f.read().split("\n")
    
    for word in lis:
        if(word not in existing_words):
            json_data[f'sign_{counter}'] = {"word":word}
            counter+=1
    with open(sign_language_data, 'w') as json_file:
        json.dump(json_data,json_file)




for key,value in json_data.items():
    print(key,value['word'])

sign_0 تعارف
sign_1 السلام
sign_2 عليكم
sign_3 رحمه
sign_4 الله
sign_5 بركاته


In [16]:
def translate(arr):
    final = []
    for word in arr:
        if word in synonyms_table:
            final.append(synonyms_table[word])
        else:
            for letter in word:
                final.append(letter)
    return final

In [18]:
# ex ; we don't have سار but we  have ذهب 
# we don't have طريق so it spelled it 
text = clean_str("السلام عليكم ورحمه الله وبركاته")
results = lemmatization(text)
translate(results)

['س',
 'ل',
 'ا',
 'م',
 'ع',
 'ل',
 'ى',
 'ر',
 'ح',
 'م',
 'ا',
 'ل',
 'ل',
 'ه',
 'ب',
 'ر',
 'ك',
 'ة']

# steaming (might be used in the future)

In [20]:
import nltk

In [21]:
# https://www.nltk.org/api/nltk.stem.isri.html
obj = nltk.stem.isri.ISRIStemmer()


In [29]:
for i in ["الخير"]:
    print(obj.stem(i))

خير


In [17]:
for i in ["الاستنشاق","المحبة","الرحمة","يذهب","السما"]:
    print(obj.stem(i))

نشق
حبة
رحم
ذهب
سما
